In [2]:
import sys
sys.path.append("../")
from tensorflow import keras
model = keras.models.load_model('outputs/sentence_level_preprocess-checkpoints')

In [3]:
from src.preprocess.word_embedding_features import sentence_level_preprocess

In [4]:
import pandas as pd
df = pd.read_csv("../data/data.csv",nrows=5000)

In [11]:
df["author"] = pd.Categorical(df["author"]).codes

In [14]:
df["author"].value_counts()[:5]

2401    32
1566    22
1250    18
2512    16
2199    16
Name: author, dtype: int64

In [17]:
df = df[(df["author"] == 2512) | (df["author"]==2199)][["author","Text"]]

In [20]:
df = df.reset_index()

,index,author,Text
0,1075,2512,\n# The Farpool: Union\n\n### Published by Ph...
1,1293,2199,\n### Baiting & Fishing\n\nby\n\nMeredith Rae...
2,1531,2199,\n### MARATHON NIGHTMARE\n\nby\n\nMeredith Ra...
3,1541,2199,\n### HELL IN TEXAS\n\nby\n\nMeredith Rae Mor...
4,1631,2199,\n### After The Fire\n\nby\n\nMeredith Rae Mo...
5,1897,2512,\n# The Farpool\n\n### Published by Philip Bo...
6,1975,2199,\nSECOND CHANCES\n\nby\n\nMeredith Rae Morgan...
7,1989,2512,\n# The Farpool: Marauders of Seome\n\n### Pu...
8,2119,2199,\n### MOUNTAIN PEOPLE\n\nby\n\nMeredith Rae M...
9,2124,2512,\n# The Farpool: Exodus\n\n### Published by P...


In [25]:
import re
df["Text"] = df["Text"].apply(lambda text: re.sub('[^A-Za-z0-9\n\s]+', '', text).replace("\n\n","\n"))

In [27]:
df["Text"].apply(len)

0      665880
1      499046
2      423356
3      385826
4      399960
5      948395
6      325359
7      775720
8      499511
9      776185
10     658507
11     538212
12     934305
13     360226
14     460781
15     412683
16     821211
17     435963
18     656753
19     767395
20     904281
21     337542
22     315793
23     314781
24     444524
25     640300
26     799786
27     361503
28    1402850
29     396691
30     719922
31    2217034
Name: Text, dtype: int64

In [ ]:
X = df["Text"].apply(sentence_level_preprocess)

downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained em

downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained embedding model.
this may take a while...
downloading pretrained em

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.2)

In [ ]:
history = model.fit(x=X_train, y=y_train, epochs=30000, shuffle=True,
                    batch_size=200, validation_data=(X_val, y_val),

In [ ]:
model.predict().argmax(axis=-1).flatten()